In [3]:
!pip install numpy

In [4]:
import math
import numpy as np
import operator
import matplotlib.pyplot as plt

event_titles_map = {}
event_client_map={}
client_event_map={}
client_avg_rating = {}
cache = {}



In [5]:
def read_event_title(fname):   
    with open(fname) as f:
        for line in f.readlines():
            parts = [x.strip() for x in line.split(',')]
            event_id = int(parts[0])
            event_titles_map[event_id] = parts[2] + "("+ parts[1]+")"    

def get_ratings_map(fname):
    with open(fname) as f:
        for line in f.readlines():
            parts = [x.strip() for x in line.split(',')]
            event_title_id = int(float(parts[0]))
            client_id = int(float(parts[1]))
            rating = float(parts[2])
            if client_id not in client_event_map:
                client_event_map[client_id]={}
            client_event_map[client_id][event_title_id] = rating
            
            if event_title_id not in event_client_map:
                event_client_map[event_title_id] = []
            event_client_map[event_title_id].append(client_id)


In [6]:
def get_client_avg_rating():
    for client in client_event_map:
        sum = 0
        i=0
        for event in client_event_map[client]:
            sum = sum + float(client_event_map[client][event])
            i = i + 1 
        avg = sum/i
        client_avg_rating[client]=avg


In [7]:
event_titles_filename="event_titles.txt"
# event_titles_filename.head()
ratings_filename="ratings.txt"
# ratings_filename.head()

read_event_title(event_titles_filename)
get_ratings_map(ratings_filename)
get_client_avg_rating()


FileNotFoundError: [Errno 2] No such file or directory: 'event_titles.txt'

In [ ]:
def calculate_accuracy():
    correct_predictions = 0
    total_predictions = 0
    for client in client_event_map:
        for event in client_event_map[client]:
            total_predictions += 1
            predicted_rating = predict_rating(client, event)
            if predicted_rating == client_event_map[client][event]:
                correct_predictions += 1
    accuracy = (correct_predictions / total_predictions) * 100
    return accuracy


In [ ]:
def predict_rating(client, event):
    if client not in cache:
        client_correlation = get_client_corr(client)
        temp_rating = 0
        for other_client in client_correlation:
            if event in client_event_map[other_client]:
                temp_rating += client_correlation[other_client] * (client_event_map[other_client][event] - client_avg_rating[other_client])
        predicted_rating = client_avg_rating[client] + temp_rating
        cache[client] = predicted_rating
    else:
        predicted_rating = cache[client]
    return predicted_rating


In [ ]:
def get_client_corr(active_client):
    client_correlation = {}
    for client in client_event_map:
        if client != active_client:
            nominator = 0
            sum_vaj_diff = 0
            sum_vij_diff = 0
            for event in client_event_map[active_client]:
                if event in client_event_map[client]:                
                    nominator +=  (client_event_map[active_client][event] - client_avg_rating[active_client]) * (client_event_map[client][event] - client_avg_rating[client])
                    sum_vaj_diff +=  np.power(client_event_map[active_client][event] - client_avg_rating[active_client], 2)
                    sum_vij_diff +=  np.power(client_event_map[client][event] - client_avg_rating[client], 2)
            denominator= np.sqrt(sum_vaj_diff * sum_vij_diff)  
            if denominator != 0:
                client_correlation[client] = nominator / denominator
    return client_correlation


In [ ]:
def recommendation(active_client, K):
    if active_client not in cache:
        client_correlation = get_client_corr(active_client)
        predicted_rating ={}
        for event in event_titles_map:
            temp_rating = 0
            if event in event_client_map:
                for client in event_client_map[event]:
                    if client in client_correlation:
                        temp_rating +=client_correlation[client]*(client_event_map[client][event]- client_avg_rating[client])
            predicted_rating[event] = temp_rating

        predicted_rating = sorted(predicted_rating.items(), key=lambda kv: kv[1], reverse=True)
        cache[active_client] = predicted_rating
    else:
        predicted_rating= cache[active_client]
        
    recommended_events = predicted_rating[:K]
    for event in recommended_events:
        print(event_titles_map[event[0]])


In [8]:
active_client = 1392773
K = 5
recommendation(active_client, K)

NameError: name 'recommendation' is not defined

In [ ]:
# Save the trained model to model.pkl
import pickle
model = cache
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
# Assuming you have features for each event stored in a dictionary event_features_map

event_features_map = {
    # Define your event features here
}

def calculate_content_similarity(event1, event2):
    # Implement a similarity measure between two events based on their features
    # Example: Cosine similarity, Jaccard similarity, etc.
    pass

def content_based_recommendation(active_client, K):
    user_profile = client_event_map.get(active_client, {})
    recommendations = []

    for event in event_features_map:
        # Calculate similarity between the event and items the user has interacted with
        similarity_score = calculate_content_similarity(event, user_profile)
        
        # Exclude events already interacted with by the user
        if event not in user_profile:
            recommendations.append((event, similarity_score))

    # Sort recommendations based on similarity score and select top-K
    recommendations.sort(key=lambda x: x[1], reverse=True)
    top_recommendations = recommendations[:K]
    
    return top_recommendations

# Example usage
active_client = 1987434
K = 5
content_based_recommendations = content_based_recommendation(active_client, K)
